- Extraction of Sentence

In [ ]:
import pandas as pd
import Sentiment
import sqlite3
# 신조어 목록 불러오기
df = pd.read_excel('new_words_var.xlsx')
new_words = df[df.y == 1].iloc[:,1].drop_duplicates(keep='first')

# 전체 데이터 합치기
conn = sqlite3.connect('Total_Ent.db')
df1 = pd.read_sql('SELECT head FROM head', conn)
conn.close()
conn = sqlite3.connect('HP.db')
df2 = pd.read_sql('SELECT head FROM head', conn)
conn.close()
head = df1.append(df2)

# sentence 추출
analysis = Sentiment.Sentiment()
sent = analysis.extract_sent(head, new_words)
sent = pd.DataFrame.from_dict(sent, orient='index')
conn = sqlite3.connect('sent.db')
sent.to_sql('sent', conn)

- Update Sentiment Dictionary and Extraction of Sentiment Score

In [ ]:
import Sentiment
from tqdm.notebook import tqdm
import sqlite3
import pandas as pd

# Initialize
analysis = Sentiment.Sentiment()
conn = sqlite3.connect('sent.db')
sent = pd.read_sql('SELECT * FROM sent', conn)
mapping_most_related = analysis.cal_score(sent)

positive = []
negative = []
for _ in tqdm(range(10)):
    # 일정 (단어-감성score) set을 사용하여, 감성사전 score 업데이트 (학습과정)
    sentiment_result = pd.read_excel('sentiment_result.xlsx')
    positive, negative = analysis.update_score(positive, negative, sentiment_result) # 감성사전 score 업데이트
    
    # 감성사전을 이용하여 감성분석을 진행
    sentiment = pd.read_excel('sentiment.xlsx')
    analysis.sentiment_analysis(sentiment, mapping_most_related)

- Extract of keywords

In [1]:
import Sentiment
from tqdm.notebook import tqdm
import sqlite3
import pandas as pd

# Initialize
analysis = Sentiment.Sentiment()
conn = sqlite3.connect('sentiment_sent.db')
sent = pd.read_sql('SELECT * FROM sent', conn)

In [ ]:
# keyword 추출
from collections import defaultdict
length = list(range(len(sent)))
length.pop(-1)
ktemp = defaultdict(lambda:0)
rtemp = defaultdict(lambda:0)
for _ in length:
    words = {sent.iloc[_]['index']:1.0}
    keywords = analysis.most_similar(sent.iloc[_]['index'], sent.iloc[_]['0'], words)
    ktemp[sent.iloc[_]['index']] = keywords[0]
    rtemp[sent.iloc[_]['index']] = keywords[1]
pd.DataFrame.from_dict(ktemp, orient='index').to_excel('keyword.xlsx')
pd.DataFrame.from_dict(rtemp, orient='index').to_excel('rkeyword.xlsx')

training was done. used memory 0.267 Gb0.267 Gb
all cohesion probabilities was computed. # words = 1411
Create (word, contexts) matrix
  - counting word frequency from 0 sents, mem=0.267 Gb
  - scanning (word, context) pairs from 0 sents, mem=0.268 Gb
  - (word, context) matrix was constructed. shape = (275, 275)                    
  - done
training was done. used memory 0.268 Gb0.268 Gb
all cohesion probabilities was computed. # words = 2055
Create (word, contexts) matrix
  - counting word frequency from 0 sents, mem=0.268 Gb
  - scanning (word, context) pairs from 0 sents, mem=0.268 Gb
  - (word, context) matrix was constructed. shape = (169, 169)                    
  - done
training was done. used memory 0.268 Gb0.268 Gb
all cohesion probabilities was computed. # words = 2331
Create (word, contexts) matrix
  - counting word frequency from 0 sents, mem=0.267 Gb
  - scanning (word, context) pairs from 0 sents, mem=0.267 Gb
  - (word, context) matrix was constructed. shape = (84, 84)

- Top 5 keyword Extraction

In [ ]:
import pandas as pd
import ast
keyword = pd.read_excel('keyword.xlsx')
rkeyword = pd.read_excel('rkeyword.xlsx')
df = pd.merge(keyword, rkeyword, on='Unnamed: 0')

In [ ]:
from collections import defaultdict
key = defaultdict(lambda:0)
for i in range(len(df)):
    temp = [j for j in df.iloc[i,:] if type(j) == str]
    key[temp[0]] = temp[1:]
    
for k,v in key.items():
    temp = [ast.literal_eval(_) for _ in v]
    temp = sorted(temp, key=lambda _:_[-1], reverse=True)
    while len(temp) > 5:
        temp.pop(-1)
    key[k] = temp

pd.DataFrame.from_dict(key, orient='index').to_excel('key.xlsx')